# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
weather_data = pd.read_csv('weather_df.csv')
weather_data = weather_data.drop(columns=['Unnamed: 0'])

weather_data.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,mataura,72,NZ,1579921333,60,-46.19,168.86,66.99,8.01
1,hermanus,0,ZA,1579921242,96,-34.42,19.23,64.99,1.32
2,castro,1,BR,1579921137,98,-24.79,-50.01,56.61,4.47
3,nelson bay,100,NZ,1579921333,58,-41.28,173.28,75.99,1.99
4,arraial do cabo,20,BR,1579921333,77,-22.84,-46.65,66.20,3.36


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Use the Lat and Lng as locations and Humidity as the weight
locations = weather_data[['Lat', 'Lng']]
humidity = weather_data['Humidity'].astype(float)

# map
fig = gmaps.figure()

# Add Heatmap layer to map
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                            dissipating=False, max_intensity=5,
                            point_radius=1)
                                
fig.add_layer(heat_layer)

# Display heatmap
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Narrow down the cities to fit weather conditions
new_weather_df = pd.DataFrame(weather_data, columns = ['City', 'Max Temp', 'Wind Speed', 'Cloudiness'])
max_temp = (new_weather_df['Max Temp'] <= 80) & (new_weather_df['Max Temp'] > 70)
wind_speed = new_weather_df['Wind Speed'] < 10
cloudiness = new_weather_df['Cloudiness'] == 0
new_weather_df[max_temp & wind_speed & cloudiness]

,City,Max Temp,Wind Speed,Cloudiness
134,jardim,70.45,2.42,0
185,cidreira,72.99,2.13,0
209,bonthe,75.78,2.77,0
248,rocha,70.83,7.90,0
280,beterou,71.15,8.32,0
299,champerico,74.91,3.74,0
437,yorosso,70.38,9.84,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
#Store into variable named hotel_df.
hotel_df = pd.DataFrame(weather_data, columns =['City', 'Country', 'Lat', 'Lng'])
hotel_df['Hotel Name']=''
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
0,mataura,NZ,-46.19,168.86,
1,hermanus,ZA,-34.42,19.23,
2,castro,BR,-24.79,-50.01,
3,nelson bay,NZ,-41.28,173.28,
4,arraial do cabo,BR,-22.84,-46.65,


In [6]:
# Set parameters to search for hotels with 5000 meters
target_coordinates = f"{hotel_df['Lat'][0]},{hotel_df['Lng'][0]}"
target_search = "hotel"
target_radius = 5000
target_type = "hotel"

# Set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Request using params dictionary
response = requests.get(base_url, params=params)

In [8]:
hotel_df.iloc[0][0]

'mataura'

In [9]:
#creates a list to capture hotel_names to insert into dataframe
hotel_name = []

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# capture hotel coordinates
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    try:
        hotel_name.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)

In [10]:
hotel_name

['Falls Hotel',
 'Birkenhead House',
 'Hotel Buganville Express',
 'Trailways Hotel',
 nan,
 nan,
 'Esplanade Hotel',
 nan,
 'Hilton Garden Inn Kauai Wailua Bay',
 'Hotel Molino Viejo',
 'Kemping wifi hotel',
 'Hampton Inn & Suites Jamestown',
 nan,
 'Paradise First Hotel',
 'Arennas Máncora',
 nan,
 'Isles Sunset Lodge',
 "Munitsipal'naya Gostinitsa",
 nan,
 'Hotel Angmagssalik',
 'Kak Doma',
 'Hotel Las Olas',
 'Tourist House - Sablino',
 'Hotel Restaurante América',
 'Foveaux Hotel',
 'BreakFree on Cashel Christchurch',
 '百名伽藍',
 'Westmark Fairbanks Hotel & Conference Center',
 'Westmark Fairbanks Hotel & Conference Center',
 'Hilton Garden Inn Laramie',
 "Smitty's Bed and Breakfast",
 'Grand Noble Hotel',
 'Grand Dafam Bela Ternate',
 nan,
 'Six Acres Bed & Breakfast',
 'Wangjiao Hotel',
 nan,
 'The Brunswick Hotel',
 'HOTEL TARKA',
 'Scandic Bryggen',
 nan,
 'in Western',
 'Hospitality Esperance',
 'Kairos Cottage',
 'The Swan At Hay Hotel',
 nan,
 'Café de Upernavik',
 'Hotel Vis

In [11]:
#list of hotels onto locations and drops any values with na .
hotel_df['Hotel Name']= hotel_name
hotel_df.dropna()
hotel_df.to_csv('hotel_export.csv')

In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))